In [4]:
import tensorflow as tf
from keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
DATA_DIR = "/media/christian/USB STICK/Images/Subset"
seed = 10
image_width = 1920
image_height = 1090
im_shape = (image_width, image_height)

In [7]:
data_generator = ImageDataGenerator(
        validation_split=0.2,
        # width_shift_range=0.2,
        # height_shift_range=0.2,
        rescale=1./255,
        # shear_range=0.2,
        zoom_range=0.1,
        rotation_range=20,
        horizontal_flip=True,
        fill_mode='nearest'
)

In [ ]:
train_generator = data_generator.flow_from_directory(DATA_DIR, target_size=im_shape, shuffle=True, seed=seed,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="training")
validation_generator = data_generator.flow_from_directory(DATA_DIR, target_size=im_shape, shuffle=False, seed=seed,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="validation")


nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples
classes = list(train_generator.class_indices.keys())
print('Classes: '+str(classes))
num_classes  = len(classes)

In [ ]:
print(type(nb_train_samples))

In [15]:
from collections import Counter
counter = Counter(train_generator.classes)
max_val = float(max(counter.values()))
class_weights = {class_id : nb_train_samples/(num_images * num_classes) for class_id, num_images in counter.items()}
     

The code below defines the layers and the hyperparameters to be tuned, as well as the parameters for tuning the hyperparameters.

In [16]:
def model_builder(hp):
    model = tf.keras.Sequential()

    #Learning Rate
    hp_initial_learning_rate = hp.Choice('learning_rate',
                                          values=[0.01, 0.001, 0.0001])
    hp_decay_rate = hp.Float('decay_rate', min_value=0.9, max_value=0.98,
                            step=0.01)
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      hp_initial_learning_rate,
      decay_steps=100000,
      decay_rate=hp_decay_rate,
      staircase=True)

    #Conv Layers
    hp_activation = hp.Choice('activation', values=['relu', 'tanh', 'gelu'])
    hp_kernel_size = hp.Int('kernel_size', min_value=1, max_value=6, step=1)
    hp_filters = hp.Int('filters', min_value=8, max_value=72, step = 8)
    number_of_layers = hp.Int('layer_amt', min_value=1, max_value=3, step=1)
    hp_pool_size = hp.Int('pool_size', min_value=1, max_value=4, step=1)

    for layer_num in range(number_of_layers):
      model.add(tf.keras.layers.Conv2D(filters=round(hp_filters/(layer_num+1)),
                                      kernel_size=(hp_kernel_size, hp_kernel_size),
                                      activation=hp_activation,
                                      input_shape=(32, 32, 3)))
      model.add(tf.keras.layers.MaxPool2D(pool_size=(hp_pool_size, hp_pool_size), strides=None, padding='valid'))
      #Add maxpooling layers dumbass

    model.add(tf.keras.layers.Flatten())


    #Dense Layers

    hp_layer_1 = hp.Int('layer_1', min_value=50, max_value=1000, step=50)
    hp_layer_2 = hp.Int('layer_2', min_value=50, max_value=1000, step=50)


    model.add(tf.keras.layers.Dense(units=hp_layer_1, activation=hp_activation))
    model.add(tf.keras.layers.Dense(units=hp_layer_2, activation=hp_activation))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    return model


In [ ]:
import keras_tuner as kt

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='dir',
                     project_name='custom-data',
                     overwrite=True)

In [19]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
#this will stop the training if the model's val_loss does not improve over 3
#consecutive epochs

In [ ]:
tuner.search(train_generator, epochs=10, validation_data=validation_generator,
             callbacks=[TensorBoard("/tmp/tb_logs")])

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=3)
print("Best run:")
print(best_hps[0].values)
print("2nd best run:")
print(best_hps[1].values)
print("3rd best run:")
print(best_hps[2].values)

In [ ]:
model = tuner.hypermodel.build(best_hps[0])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2,
                    callbacks=[stop_early])

Best model: val_acc

In [ ]:
%load_ext tensorboard

%tensorboard --logdir /tmp/tb_logs

In [ ]:
!zip -r tensorboard_logs.zip /tmp/tb_logs